Read a list configurations from a JSON file

In [9]:
%load_ext blackcellmagic

In [10]:
import json
import libcasm.configuration as casmconfig

# read prim.json
with open("prim.json", "r") as f:
    prim = casmconfig.Prim.from_dict(json.load(f))

In [11]:
import ase
import libcasm.xtal as xtal

# Conversion function from ase.Atoms to libcasm.xtal.Structure
def casm_structure_to_ase_atoms(casm_structure: xtal.Structure) -> ase.Atoms:
    """Given `xtal.Structure`, convert it to `xtal.Structure`

    Parameters
    ----------
    casm_structure : xtal.Structure

    Returns
    -------
    ase.Atoms

    """
    if len(casm_structure.mol_type()):
        raise ValueError(
            "Error: only atomic structures may be converted using " "make_ase_atoms"
        )

    symbols = casm_structure.atom_type()
    positions = casm_structure.atom_coordinate_cart().transpose()
    cell = casm_structure.lattice().column_vector_matrix().transpose()

    return ase.Atoms(
        symbols=symbols,
        positions=positions,
        cell=cell,
        pbc=True,
    )


# Conversion function from ase.Atoms to libcasm.xtal.Structure
def ase_atoms_to_casm_structure(ase_atoms: ase.Atoms) -> xtal.Structure:
    """Given `ase.Atoms`, convert it to `xtal.Structure`

    Parameters
    ----------
    ase_atoms : ase.Atoms

    Returns
    -------
    xtal.Structure

    """

    lattice = xtal.Lattice(
        column_vector_matrix=ase_atoms.get_cell().transpose(),
    )
    atom_coordinate_frac = ase_atoms.get_scaled_positions().transpose()
    atom_type = ase_atoms.get_chemical_symbols()

    return xtal.Structure(
        lattice=lattice,
        atom_coordinate_frac=atom_coordinate_frac,
        atom_type=atom_type,
    )

In [18]:
import os

# read config_list.json -> containing a list of enumerated configurations
with open("config_list.json", "r") as f:
    data = json.load(f)

# convert the data into a configuration set
configuration_set = casmconfig.ConfigurationSet()
for d in data:
    config = casmconfig.Configuration.from_dict(
        data=d, supercells=casmconfig.SupercellSet(prim=prim)
    )
    configuration_set.add_configuration(config)


# write vasp input files for each of the configurations
# will be written to <casm_proj_name>/training_data/<enumuration_id>/<config_name>/calctype.<calctype_id>/
casm_proj_name = "SiGe"  # define casm_proj_name
enum_id = "occ"  # define enumeration_id
calctype_id = "default"  # define calctype_id
for config in configuration_set:
    # get the config_name
    config_name = os.path.join(config.supercell_name, config.configuration_id)

    # final path where INCAR, KPOINTS, POTCAR, POSCAR will be written
    config_calctype_dir = os.path.join(
        casm_proj_name, "training_data", enum_id, config_name, "calctype." + calctype_id
    )

    # make config_calctype_dir
    os.makedirs(config_calctype_dir, exist_ok=True)

    # Write poscar
    poscar_str = config.configuration.to_structure().to_poscar_str()
    poscar_path = os.path.join(config_calctype_dir, "POSCAR")

    print("Writing POSCAR for: ", config_name, " at ", poscar_path)
    with open(poscar_path, "w") as f:
        f.write(poscar_str)


Writing POSCAR for:  SCEL1_1_1_1_0_0_0/0  at  SiGe/training_data/occ/SCEL1_1_1_1_0_0_0/0/calctype.default/POSCAR
Writing POSCAR for:  SCEL1_1_1_1_0_0_0/1  at  SiGe/training_data/occ/SCEL1_1_1_1_0_0_0/1/calctype.default/POSCAR
Writing POSCAR for:  SCEL2_2_1_1_0_1_1/0  at  SiGe/training_data/occ/SCEL2_2_1_1_0_1_1/0/calctype.default/POSCAR
Writing POSCAR for:  SCEL2_2_1_1_0_0_1/0  at  SiGe/training_data/occ/SCEL2_2_1_1_0_0_1/0/calctype.default/POSCAR
Writing POSCAR for:  SCEL3_3_1_1_0_2_2/0  at  SiGe/training_data/occ/SCEL3_3_1_1_0_2_2/0/calctype.default/POSCAR
Writing POSCAR for:  SCEL3_3_1_1_0_2_2/1  at  SiGe/training_data/occ/SCEL3_3_1_1_0_2_2/1/calctype.default/POSCAR
Writing POSCAR for:  SCEL3_3_1_1_0_0_2/0  at  SiGe/training_data/occ/SCEL3_3_1_1_0_0_2/0/calctype.default/POSCAR
Writing POSCAR for:  SCEL3_3_1_1_0_0_2/1  at  SiGe/training_data/occ/SCEL3_3_1_1_0_0_2/1/calctype.default/POSCAR
Writing POSCAR for:  SCEL3_3_1_1_0_2_1/0  at  SiGe/training_data/occ/SCEL3_3_1_1_0_2_1/0/calctyp